In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
## Imports and environment variables 
import torch, wandb, os, pandas as pd 
from travis_attack.utils import set_seed, set_session_options, setup_logging, setup_parser, update_config_with_parsed_arguments, resume_wandb_run, display_all, print_important_cfg_vars
from travis_attack.config import Config
from travis_attack.models import prepare_models, get_optimizer
from travis_attack.data import ProcessedDataset
from travis_attack.trainer import Trainer
from travis_attack.insights import (postprocess_df, create_and_log_wandb_postrun_plots, get_training_dfs)
from fastcore.basics import in_jupyter

import logging 
logger = logging.getLogger("run")

import warnings
warnings.filterwarnings("ignore", message="Passing `max_length` to BeamSearchScorer is deprecated")  # we ignore the warning because it works anyway for diverse beam search 

In [ ]:
from nbdev.export import notebook2script
notebook2script()

!jupyter nbconvert \
    --TagRemovePreprocessor.enabled=True \
    --TagRemovePreprocessor.remove_cell_tags="['hide']" \
    --TemplateExporter.exclude_markdown=True \
    --to python "run.ipynb"

Converted 00_utils.ipynb.
Converted 02_tests.ipynb.
Converted 03_config.ipynb.
Converted 07_models.ipynb.
Converted 10_data.ipynb.
Converted 20_trainer.ipynb.
Converted 25_insights.ipynb.
Converted Untitled.ipynb.
Converted Untitled1.ipynb.
Converted baselines.ipynb.
Converted baselines_analysis.ipynb.
Converted index.ipynb.
Converted pp_eval_baselines.ipynb.
Converted run.ipynb.
Converted show_examples.ipynb.
Converted statistical_tests.ipynb.
Converted test_grammar_options.ipynb.
Converted test_pp_model.ipynb.
[NbConvertApp] Converting notebook run.ipynb to python


In [ ]:
cfg = Config()  # default values
if not in_jupyter():  # override with any -- options when running with command line
    parser = setup_parser()
    newargs = vars(parser.parse_args())
    cfg = update_config_with_parsed_arguments(cfg, newargs)
if cfg.use_small_ds:  cfg = cfg.small_ds()
set_seed(cfg.seed)
set_session_options()
setup_logging(cfg, disable_other_loggers=True)
vm_tokenizer,vm_model,pp_tokenizer,pp_model,ref_pp_model,sts_model,nli_tokenizer,nli_model,cola_tokenizer,cola_model,cfg = prepare_models(cfg)
optimizer = get_optimizer(cfg, pp_model)
ds = ProcessedDataset(cfg, vm_tokenizer, vm_model, pp_tokenizer, sts_model, load_processed_from_file=False)

travis_attack.data: INFO     Will load dataset financial with use_small_ds set to True
travis_attack.data: INFO     Will load dataset financial with use_small_ds set to True
travis_attack.data: INFO     Will load dataset financial with use_small_ds set to True


In [ ]:
cfg.wandb['mode'] = 'online'
trainer = Trainer(cfg, vm_tokenizer,vm_model,pp_tokenizer,pp_model,ref_pp_model,sts_model,nli_tokenizer,nli_model,cola_tokenizer,cola_model, optimizer,
         ds)
print_important_cfg_vars(cfg)
trainer.train()

{'pp_name': 'prithivida/parrot_paraphraser_on_T5',
 'dataset_name': 'financial',
 'sts_name': 'sentence-transformers/paraphrase-MiniLM-L12-v2',
 'nli_name': 'howey/electra-small-mnli',
 'cola_name': 'textattack/albert-base-v2-CoLA',
 'vm_name': 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis',
 'seed': 420,
 'use_small_ds': True,
 'lr': 8e-05,
 'batch_size_train': 32,
 'batch_size_eval': 16,
 'acc_steps': 2,
 'n_train_epochs': 2,
 'eval_freq': 1,
 'reward_fn': 'reward_fn_contradiction_and_letter_diff',
 'reward_clip_max': 4,
 'reward_vm_multiplier': 12,
 'sts_threshold': 0.8,
 'acceptability_threshold': 0.5,
 'contradiction_threshold': 0.2,
 'pp_letter_diff_threshold': 30,
 'reward_penalty_type': 'kl_div',
 'kl_coef': 0.25,
 'ref_logp_coef': None,
 'min_pp_length': 2,
 'max_pp_length': 48,
 'n_eval_seq': 32,
 'decode_method_train': 'sample',
 'decode_method_eval': 'sample',
 'gen_params_train': {'do_sample': True, 'temperature': 1, 'top_p': 0.95},
 'gen_params_eval':

%_of_tokens_above_prob_1/vocab_size,▁███
%_of_tokens_above_prob_1e-05,▁███
%_of_tokens_above_prob_1e-06,▁███
%_of_tokens_above_prob_1e-07,▁███
%_of_tokens_above_prob_1e-08,▁███
%_of_tokens_above_prob_1e-09,▁███
acc_batch_n_examples,▁▁▁▁
acc_num,▁█▁█
acceptability_scores-mean-test,█▁
acceptability_scores-mean-train,▆▁█▆▆
acceptability_scores-mean-valid,▁█▄▇▄


travis_attack.trainer: INFO     Launching initial eval run: train
travis_attack.trainer: INFO     Launching initial eval run: train
travis_attack.trainer: INFO     Launching initial eval run: train


travis_attack.trainer: INFO     Launching initial eval run: valid
travis_attack.trainer: INFO     Launching initial eval run: valid
travis_attack.trainer: INFO     Launching initial eval run: valid


travis_attack.trainer: INFO     Epoch: 0. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 0. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 0. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000


...

travis_attack.trainer: INFO     Now on epoch 1 of 2
travis_attack.trainer: INFO     Now on epoch 1 of 2
travis_attack.trainer: INFO     Now on epoch 1 of 2


travis_attack.trainer: INFO     Epoch: 1. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 1. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 1. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000


travis_attack.trainer: INFO     Now on epoch 2 of 2
travis_attack.trainer: INFO     Now on epoch 2 of 2
travis_attack.trainer: INFO     Now on epoch 2 of 2


travis_attack.trainer: INFO     Epoch: 2. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 2. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 2. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000


travis_attack.trainer: INFO     Evaluating test set with best model at path : ../model_checkpoints/travis_attack/vibrant-oath-130/model_0.pt
travis_attack.trainer: INFO     Evaluating test set with best model at path : ../model_checkpoints/travis_attack/vibrant-oath-130/model_0.pt
travis_attack.trainer: INFO     Evaluating test set with best model at path : ../model_checkpoints/travis_attack/vibrant-oath-130/model_0.pt


In [ ]:
%debug


> /data/tproth/travis_attack/travis_attack/utils.py(308)append_df_to_csv()
    306     if not os.path.isfile(path):   df.to_csv(path, mode='a', index=False)  # create with header if not exists
    307     elif len(df.columns) != len(pd.read_csv(path, nrows=1).columns):
--> 308         raise Exception("Columns do not match. Dataframe has " + str(len(df.columns)) + " columns. CSV file has " + str(len(pd.read_csv(path, nrows=1).columns)) + " columns.")
    309     elif not (df.columns == pd.read_csv(path, nrows=1).columns).all():
    310         raise Exception("Columns and column order of dataframe and csv file do not match.")

ipdb> df.columns
Index(['datetime_run', 'run_name', 'dataset_name', 'split', 'epoch', 'seed',
       'decode_method_train', 'decode_method_eval', 'reward_fn',
       'reward_clip_max', 'reward_clip_min', 'reward_base',
       'reward_vm_multiplier', 'sts_threshold', 'contradiction_threshold',
       'pp_letter_diff_threshold', 'pp_name', 'sts_name', 'nli_name',
 

In [ ]:
#df = pd.read_csv(cfg.path_results + "run_results.csv")
#display_all(df)

In [ ]:
# df = pd.read_csv(f'{cfg.path_run}training_step.csv')
# #display_all(df.query('idx==1'))
# df.columns

In [ ]:
#trainer.run.finish()

In [ ]:
# # ## TO RESUME RUN
# cfg = Config()
# cfg.run_id = 'b9r88lmz'
# cfg.run_name = "fresh-cosmos-151"
# cfg.path_run = f"{cfg.path_checkpoints}{cfg.run_name}/"
# run = resume_wandb_run(cfg)


In [ ]:
# df_d = get_training_dfs(cfg.path_run, postprocessed=False)
# for k, df in df_d.items(): 
#     df_d[k] = postprocess_df(df, filter_idx=None, num_proc=1)
#     df_d[k].to_pickle(f"{cfg.path_run}{k}_postprocessed.pkl")    
# create_and_log_wandb_postrun_plots(df_d)
trainer.run.finish()
#run.finish()

%_of_tokens_above_prob_1/vocab_size,▁▁
%_of_tokens_above_prob_1e-05,▁▁
%_of_tokens_above_prob_1e-06,▁▁
%_of_tokens_above_prob_1e-07,▁▁
%_of_tokens_above_prob_1e-08,▁▁
%_of_tokens_above_prob_1e-09,▁▁
acc_batch_n_examples,▁▁
acc_num,▁█
acceptability_scores-mean-test,▁█
acceptability_scores-mean-train,▄█▁
acceptability_scores-mean-valid,▁█▇
